tl;dr -- `lmt` needs there to be _either_:
- markdown after the last file piece
- two newline characters

In [22]:
!git clone https://github.com/driusan/lmt

fatal: destination path 'lmt' already exists and is not an empty directory.


In [23]:
%cd lmt

/content/lmt


In [24]:
%%capture
!apt install golang

In [25]:
#!go build main.go
#!mv main lmt
!go build

In [26]:
import os
os.environ['PATH'] = f'{os.environ["PATH"]}:/content/lmt'

In [27]:
%cd /content

/content


In [34]:
%%writefile test.go.md
# Thing

```go test.go
package main

import (
	fmt
)
```

Overwriting test.go.md


In [35]:
!lmt test.go.md

In [36]:
!cat test.go

cat: test.go: No such file or directory


In [57]:
%%writefile test2.go.md
# Thing

```go test2.go
package main

import (
	fmt
)
```


Overwriting test2.go.md


In [58]:
!lmt test2.go.md

In [59]:
!cat test2.go

cat: test2.go: No such file or directory


In [60]:
%%writefile test3.go.md
# Thing

```go test3.go
package main

import (
	fmt
)
```
# Blah

Writing test3.go.md


In [61]:
!lmt test3.go.md
!cat test3.go

//line test3.go.md:4
package main

import (
	fmt
)


In [62]:
%%writefile test4.go.md
# Thing

```go test4.go
package main

import (
	fmt
)
```



Writing test4.go.md


In [63]:
!lmt test4.go.md
!cat test4.go

//line test4.go.md:4
package main

import (
	fmt
)
